In [140]:
#importamos librerias
import pandas as pd
import numpy as np
#!pip install clean-text[gpl]
from cleantext import clean
import re
from unidecode import unidecode

#Visualization
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from matplotlib import pyplot
from matplotlib import rcParams


####Para visualizar numero de adverbios,sustantivos, adjetivos...etc
from yellowbrick.text import PosTagVisualizer


#nltk
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


#sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import precision_score, recall_score
from sklearn import metrics
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold


#importance after training
import statsmodels.api as sm

###modelos
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression



#BERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

#used in count stopwords
from collections import defaultdict
#used to count words and use in RNN
from collections import Counter

#RNN 
#from keras.models import Sequential
#from keras.layers import Embedding, LSTM, Dense, Dropout,SimpleRNN
#from keras.optimizers import Adam
#from keras import regularizers





[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [136]:
class TransformYoutubeComments:
    def __init__(self, excel_filename):
        """"We only need 1 input, the excel with the comments of youtube."""
        self.excel_filename = excel_filename
        self.every_comment = pd.DataFrame()
    
    def read_excel_sheets(self, sheet_names):
        """"As we have the comment of every video in different pages, we read every 
                 page and append in a single dataframe calls every_comment"""
        
        excel_file = pd.ExcelFile(self.excel_filename)
        
        self.every_comment = pd.concat(
            [pd.read_excel(excel_file, sheet_name=sheet_name) for sheet_name in sheet_names],
            ignore_index=True
        )
            
    def transform_data(self):
        """"This function transform the data that has previously been read. Here we add the transformation that 
                            its 100% required to do in our dataset."""
        
        self.every_comment.dropna(subset=['Comment'], inplace=True)
        self.every_comment.dropna(axis=0, how='all', inplace=True)
        self.every_comment = self.every_comment[['Comment','Likes']]
        #all text to lower
        self.every_comment['Comment'] = self.every_comment['Comment'].str.lower()
        #Clean http comments
        mask = self.every_comment['Comment'].str.contains('http', na=False)
        self.every_comment = self.every_comment[~mask]
        
        #Clean <br> in comments
        mask = self.every_comment['Comment'].str.contains('<br>', na=False)
        self.every_comment = self.every_comment[~mask]
        
        #Clean <a href in comments
        mask = self.every_comment['Comment'].str.contains('<a href', na=False)
        self.every_comment = self.every_comment[~mask]
        
        #Drop <b> and </b> in comments
        self.every_comment['Comment'] = self.every_comment['Comment'].replace('<b>', '')
        self.every_comment['Comment'] = self.every_comment['Comment'].replace('</b>', '')
        
        #Dude the behaviour of API Youtube, the letter ñ of spanish vocabulary have been read as n.
        self.every_comment['Comment'] = self.every_comment['Comment'].replace('ano', 'año')
        
        #Drop question marks,commas...etc
        self.every_comment['Comment'] = self.every_comment['Comment'].replace(to_replace=r'[^\w\s]', value='', regex=True).str.lower()
        #We generate the column we want to predict
        self.every_comment['predict'] = (self.every_comment['Likes'] >= 20).astype(int)
        
        self.every_comment = self.every_comment.reset_index(drop=True)
        self.every_comment = self.every_comment[:60]
        
    @classmethod
    def clean_emoji(cls,text,no_emoji=False):
        "We can drop emojis with this transformation if required"
        text = text.apply(lambda x: clean(x, no_emoji=True))
        return text
    
    def emoji(self):
        self.every_comment['Comment'] = self.clean_emoji(self.every_comment['Comment'])
        
    @classmethod
    def clean_accents(cls,text):
        "We can drop accents with this transformation if required"
        text = text.dropna().reset_index(drop=True) 
        text= text.apply(lambda x: x.encode('utf-8').decode('utf-8'))
        return text
    
    def accents(self):
        self.every_comment['Comment'] = self.clean_accents(self.every_comment['Comment'])
        
        
    @classmethod
    def clean_symbols(cls,text):
        "We can clean symbols with this transformation if required"
        text = text.replace(to_replace=r'[^\w\s]', value='', regex=True).str.lower()

    def symbols(self):
        "Applied clean_symbols"
        self.every_comment['Comment'] = self.clean_symbols(self.every_comment['Comment'])
        
        
    @classmethod
    def generate_nltk(cls, dataframe):
        """"Here generate nltk sentiment column """
        sia = SentimentIntensityAnalyzer()
        dataframe['sentiment_nltk'] = dataframe['Comment'].apply(lambda x: sia.polarity_scores(x)['compound'])
        return dataframe
    
    @classmethod
    def generate_bert(cls,review):
        """"Here generate bert sentiment column """
        tokens = tokenizer.encode(review, return_tensors='pt')
        result = model(tokens)
        return int(torch.argmax(result.logits))+1
    

    def generate_sentiment(self,use_nltk=True,use_bert=False):
        """"Only when its true you can use the transformation. Default BERT
                        its False because time consuming."""
        if use_nltk:
            self.every_comment = self.generate_nltk(self.every_comment)
        if use_bert:
            self.every_comment['sentiment_BERT'] = self.every_comment['Comment'].apply(lambda x: self.generate_bert(x[:512]))
            
    
    @classmethod
    def clean_stopwords(cls,text):
        "We can clean stopwords with this transformation if required"
        stop_words = set(stopwords.words('spanish'))
        text = text.apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
        return text
    
    def stopwords(self):
        "Applied stopwords"
        self.every_comment['Comment'] = self.clean_stopwords(self.every_comment['Comment'])


#Attemp to do new class for all next functions 

def process_text_data(dataframe, max_length=20, test_size=0.2):
    text = dataframe.Comment
    counter = Counter()
    for i in text.values:
        for word in str(i).split():
            counter[word] += 1
    num_words = len(counter)

    train_size = int(dataframe.shape[0] * (1 - test_size))
    train_sentences = dataframe.Comment[:train_size]
    train_labels = dataframe.predict[:train_size]

    test_sentences = dataframe.Comment[train_size:]
    test_labels = dataframe.predict[train_size:]

    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(train_sentences)

    train_sequences = tokenizer.texts_to_sequences(train_sentences)
    test_sequences = tokenizer.texts_to_sequences(test_sentences)

    train_padded = pad_sequences(
        train_sequences, maxlen=max_length, padding="post", truncating="post"
    )

    test_padded = pad_sequences(
        test_sequences, maxlen=max_length, padding="post", truncating="post"
    )

    return train_padded, train_labels, test_padded, test_labels, num_words


def scale_sentiments(dataframe,column='sentiment_BERT'):
    minimo = dataframe[column].min()
    maximo = dataframe[column].max()
    
    dataframe[column] = ((dataframe[column] - minimo) / (maximo - minimo)) * 2 - 1

    return dataframe

def apply_smote_simplernn(X_train, y_train, sampling_strategy=0.5, k_neighbors=5, random_state=42):
    sm = SMOTE(sampling_strategy=sampling_strategy, k_neighbors=k_neighbors, random_state=random_state)
    X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
    return X_resampled, y_resampled

def model_rnn(x_train,y_train,x_test,y_test,learning_rate=0.0003):
    model = Sequential()

    model.add(SimpleRNN(units=64, input_shape=(None, 1), return_sequences=True))
    model.add(Dropout(rate=0.3))
    model.add(SimpleRNN(units=32, return_sequences=True))
    model.add(Dropout(rate=0.2))
    model.add(SimpleRNN(units=16, return_sequences=False))
    model.add(Dropout(rate=0.3))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', 'AUC'])
    history = model.fit(x_train, y_train, epochs=200, validation_data=(x_test, y_test), verbose=1)
    return model,history



def model_lstm(x_train,y_train,x_test,y_test,learning_rate=0.0003):
    model = Sequential()
    model.add(Embedding(num_words, 32, input_length=22)) #22 marca la cantidad de columnas que hay en x_train x_test
    model.add(LSTM(32, dropout=0.2, return_sequences=True))  # Primera capa LSTM adicional
    model.add(LSTM(16, dropout=0.2, return_sequences=True))  # Segunda capa LSTM adicional
    model.add(LSTM(8, dropout=0.2))  # Capa LSTM original
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), verbose=1)
    return model,history


def metrics(y_test,y_pred):
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)


    print(f'Accuracy: {round(precision*100,2)}%')
    print(f'recall: {round(recall*100,2)}%')
    print(f'f1_score: {round(f1*100,2)}%')


def confusion_matrix(model,x_test,y_test,sensibility=0.5):
    # Hacer predicciones con el modelo entrenado
    y_pred = (model.predict(x_test) > sensibility).astype(int)

    # Calcular la matriz de confusión
    CM = metrics.confusion_matrix(y_test, y_pred)

    # Visualizar la matriz de confusión
    plt.figure(figsize=(9,9))
    sns.heatmap(CM, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Matriz de confusión', size = 15)
    plt.show()

    
def training_curve(history):
    """" pendiente de saber que history tengo que cambiar """
    # Plot accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()



def frase_a_predecir(texto):
    #Divide la cadena de texto en una lista de palabras
    palabras = texto.split()

    #Convierte la lista de palabras en una secuencia de índices de palabras
    secuencia = tokenizer.texts_to_sequences([palabras])[0]

    #Ajusta la longitud de la secuencia a la longitud máxima deseada
    frase = pad_sequences(
        [secuencia], maxlen=max_length, padding="post", truncating="post"
        )

    frase = pd.DataFrame(frase)

                    #####################
                    #    Hago nltk      #
                    #####################
                    
    sia = SentimentIntensityAnalyzer()

    def get_sentiment(text):
        sentiment = sia.polarity_scores(text)
    return sentiment['compound']

    #Analizo el sentimiento de mi dataframe y creo la columna sentiment_nltk
    frase['sentiment_nltk'] = get_sentiment(texto)

                    #####################
                    #    Hago BERT      #
                    #####################

    tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

    model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

    def sentiment_score(review):
        tokens = tokenizer.encode(review, return_tensors='pt')
        result = model(tokens)
    return int(torch.argmax(result.logits))+1

    texto['sentiment_BERT'] =  sentiment_score(texto)


    return frase




In [137]:
filename = 'VELADA_2.xlsx'
sheet_names = ['ALL', 'BUSTA', 'LUZU', 'VIRUZ', 'ARI', 'CARO']


llamada = TransformYoutubeComments(excel_filename=filename)
llamada.read_excel_sheets(sheet_names)
llamada.transform_data()
llamada.emoji()
#llamada.accents()
llamada.generate_sentiment()
llamada.stopwords()

In [139]:
df = llamada.every_comment

In [ ]:
texto = 'dale like para que ibai participe en la siguiente velada'

frase = frase_a_predecir(texto)

model.predict(frase)